In [40]:
import os

os.chdir("/home/nilanj/PyCharmProjects/ML/Chicken-diesease-classification")
%pwd

'/home/nilanj/PyCharmProjects/ML/Chicken-diesease-classification'

In [41]:
#update entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    
 

In [42]:
# update configuration

from cnnClassifire.constants import *
from cnnClassifire.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [53]:
# create component


import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        print(f"config.tensorboard_root_log_dir:{self.config.tensorboard_root_log_dir}, {type(self.config.tensorboard_root_log_dir)}")

        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        print(f"config.checkpoint_model_filepath:{self.config.checkpoint_model_filepath}, {type(self.config.checkpoint_model_filepath)}")
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            mode='min',
            verbose=1
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [54]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-08-23 20:42:29,213:INFO:common:yaml file: config/config.yaml loaded successfully]
[2023-08-23 20:42:29,218:INFO:common:yaml file: params.yaml loaded successfully]
[2023-08-23 20:42:29,220:INFO:common:created directory at: artifacts]
[2023-08-23 20:42:29,221:INFO:common:created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2023-08-23 20:42:29,223:INFO:common:created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
config.tensorboard_root_log_dir:artifacts/prepare_callbacks/tensorboard_log_dir, <class 'pathlib.PosixPath'>
config.checkpoint_model_filepath:artifacts/prepare_callbacks/checkpoint_dir/model.h5, <class 'pathlib.PosixPath'>
